In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
importlib.reload(kgs)
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


In [ ]:
#%%pyinstrument
#kgs.profiling=True
# %matplotlib widget
#kgs.enable_module_profiling(kgs)
import pack_ga3
import pack_runner
fastMode = False
kgs.debugging_mode = 1

runner = pack_ga3.baseline_symmetry_180()
runner.ga.N = 2
runner.ga.ga_base.N_trees_to_do = 200
runner.n_generations = 1
runner.diagnostic_plot = True
runner.seed = 0
runner.plot_every = 1
runner.ga.do_legalize = False

runner.run()

In [ ]:
pop = runner.ga.ga_list[0].champions[0]
pop.merge(runner.ga.ga_list[1].champions[0])

In [ ]:
import pack_ga3
import pack_move
import pack_vis_sol
importlib.reload(pack_move)
move = runner.ga.ga_base.move.moves[-1]
move = move[0]
move.jitter = 0.
#move = pack_move.CrossoverStripe()
#move.distance_function = 'square90'
#move.decouple_mate_location = True
#move.distance_function = 'stripe'
move.min_N_trees = 25
print(move)
generator = cp.random.default_rng(seed=12345)
for _ in range(20):
    pop_here = copy.deepcopy(pop)
    move.do_move_vec(pop_here, cp.array([0]), pop_here.genotype, cp.array([1]), generator)
    pop_here.genotype.canonicalize()
    #print(move_descriptor)
    fig,ax = plt.subplots(1,3,figsize=(18,6))
    plt.sca(ax[0])
    pack_vis_sol.pack_vis_sol(pop.genotype, ax=ax[0], solution_idx=0)
    plt.sca(ax[1])
    pack_vis_sol.pack_vis_sol(pop_here.genotype, ax=ax[1], solution_idx=0)
    plt.sca(ax[2])
    pack_vis_sol.pack_vis_sol(pop.genotype, ax=ax[2], solution_idx=1)
    fig,ax = plt.subplots(1,3,figsize=(18,6))
    plt.sca(ax[0])
    pack_vis_sol.pack_vis_sol(pop.genotype.convert_to_phenotype(), ax=ax[0], solution_idx=0)
    plt.sca(ax[1])
    pack_vis_sol.pack_vis_sol(pop_here.genotype.convert_to_phenotype(), ax=ax[1], solution_idx=0)
    plt.sca(ax[2])
    pack_vis_sol.pack_vis_sol(pop.genotype.convert_to_phenotype(), ax=ax[2], solution_idx=1)
    plt.pause(0.001)
    # pack_vis.visualize_tree_list(tree_list, ax=ax[0])
    # plt.sca(ax[1])
    # tree_list = kgs.TreeList()
    # tree_list.xyt = pop_here.genotype.xyt[0].get()
    # pack_vis.visualize_tree_list(tree_list, ax=ax[1])
    # plt.sca(ax[2])
    # tree_list = kgs.TreeList()
    # tree_list.xyt = pop_here.genotype.xyt[1].get()
    # pack_vis.visualize_tree_list(tree_list, ax=ax[2])
    # plt.pause(0.001)